In [1]:
import pandas as pd
import numpy as np

In [4]:
item_info = pd.read_csv('../rawdata/item_info.csv')
df = pd.read_csv('../rawdata/2014-01-01.csv', encoding = 'cp949')

In [7]:
item_dict =item_info.set_index('pum_id').to_dict()['pum_name']

In [9]:
df['pum_name'] =df['pum_id'].map(item_dict)

In [10]:
df

,collect_day,good_id,pum_id,pum_name,good_name,sales_price,discount_price,benifit
0,2014-01-01,3.740000e+03,E061100,습기제거제,알뜰상품_제습제_4입,4650.0,NaN,NaN
1,2014-01-01,6.847000e+03,L012040,치약,비손_예스미백치약_120G,770.0,NaN,NaN
2,2014-01-01,1.566640e+05,L012080,화장지,해피트리_3겹천연펄프화장지_35M*12R(S),5800.0,NaN,NaN
3,2014-01-01,2.690740e+05,L012180,NaN,리스테린 쿨 민트/후레쉬버스트/내추럴 시트러스/티쓰앤껌 750mlx2개 초특가,7800.0,NaN,NaN
4,2014-01-01,3.565480e+05,C021010,아동화,Allie Ballet Flat,57140.0,NaN,NaN
...,...,...,...,...,...,...,...,...
83452,2014-01-01,2.146854e+09,A017130,NaN,호박고구마보다 맛있는 정품 꿀고구마 5kg 9900원/10kg 15900원/정품 밤...,9900.0,NaN,NaN
83453,2014-01-01,2.146984e+09,L012010,면도기,쉬크 쿼트로4티타늄면도기,10900.0,NaN,NaN
83454,2014-01-01,2.147114e+09,A015010,참기름,★의성 노현국참기름★국내산 명품참기름★참기름/들기름/생들기름/참깨/선물세트/어버이날...,40000.0,NaN,NaN
83455,2014-01-01,2.147285e+09,A019060,된장,제주 친환경 무농약 콩 제주콩된장 950g (재래식된장/발효식품) [W022337],36230.0,NaN,NaN
